In [1]:
# Import necessary modules
import cvxpy as cvx
import numpy as np

In [2]:
def sum_elem_product(A,B):
    return cvx.sum_entries(cvx.mul_elemwise(A, B))

def col_vec_4elem(a,b,c,d):
    return np.matrix([[a],[b],[c],[d]])

In [7]:
# Define anchors
anchors = np.matrix([
    [ 1, 0],
    [-1, 0],
    [ 0, 1.4]
])

# Number of dimensions
n = 2

# Sensor locations
# sensor_location_in = [0.44, 0.25] # Sensor inside convex hull
# sensor_location_out = [1, 1]      # Sensor outside convex hull

# case 1: sensor_location_both_in
s1 = np.matrix([
    [ 0, .1],
    [0, 0.5]]) 

# case 2: sensor_location_one_in_one_out_in_anchors
s1 = np.matrix([
    [ 0, .1],
    [-.15, 1.8]])

# case 3: sensor_location_one_in_one_out_one_not_in_anchors
s3 = np.matrix([
    [ 0, .1],
    [-.5, 10]])

# case 4: one in one out, not in convex hull of anchors
s3 = np.matrix([
    [ 0, .1],
    [0.05, 10]])


# case 4: not in each others' convex hull
s4 = np.matrix([
    [ -.5, .5],
    [.5, 0.5]])

# Distances from anchors

d0 = list(map(lambda a: np.linalg.norm(s1[0, :] - a), anchors))
d1 = list(map(lambda a: np.linalg.norm(s1[1, :] - a), anchors))
d_both_sensor = np.linalg.norm(s1[0, :] - s1[1, :])
# d_out = list(map(lambda a: np.linalg.norm(sensor_location_out - a), anchors))

In [8]:
# Constraints will define the proper structure for Z. The optimal value will be 
# located in the top right two elements of Z.
Z = cvx.Semidef(4)

v0 = col_vec_4elem(1,0,0,0)
v1 = col_vec_4elem(0,1,0,0)
v2 = col_vec_4elem(1,1,0,0)

a0 = col_vec_4elem(anchors[0,0], anchors[0,1], -1, 0)
a1 = col_vec_4elem(anchors[1,0], anchors[1,1], -1, 0)

a2 = col_vec_4elem(anchors[1,0], anchors[1,1], 0, -1)
a3 = col_vec_4elem(anchors[2,0], anchors[2,1], 0, -1)

v3 = col_vec_4elem(0, 0, 1, -1)

constraints = [
    sum_elem_product(v0*np.transpose(v0), Z) == 1,
    sum_elem_product(v1*np.transpose(v1), Z) == 1,
    sum_elem_product(v2*np.transpose(v2), Z) == 2,
    sum_elem_product(a0*np.transpose(a0), Z) == cvx.square(d0[0]),
    sum_elem_product(a1*np.transpose(a1), Z) == cvx.square(d0[1]),
    sum_elem_product(a2*np.transpose(a2), Z) == cvx.square(d1[1]),
    sum_elem_product(a3*np.transpose(a3), Z) == cvx.square(d1[2]),
    sum_elem_product(v3*np.transpose(v3), Z) == cvx.square(d_both_sensor)
]

In [20]:
c1 = col_vec_4elem(anchors[2,0], anchors[2,1], -1, 0)
c2 = col_vec_4elem(anchors[0,0], anchors[0,1], 0, -1)
C = c1*np.transpose(c1) + c2*np.transpose(c2)
objective = cvx.Maximize(sum_elem_product(C, Z))

(4, 4)


In [27]:
c1 = col_vec_4elem(anchors[2,0], anchors[2,1], -1, 0)
c2 = col_vec_4elem(anchors[0,0], anchors[0,1], 0, -1)
C = c1*np.transpose(c1) + c2*np.transpose(c2)
objective = cvx.Minimize(sum_elem_product(C, Z))

In [ ]:
C = np.identity(4)
objective = cvx.Maximize(sum_elem_product(C, Z))

In [25]:
C = np.identity(4)
objective = cvx.Minimize(sum_elem_product(C, Z))

In [28]:
prob = cvx.Problem(objective, constraints)
result = prob.solve(solver = 'MOSEK')

# Optimal solution
x_1_star = [Z[0,2].value, Z[1,2].value]
x_2_star = [Z[0,3].value, Z[1,3].value]

print('Results for Sensor Inside of Convex Hull of Anchors')
print('---------------------------------------------------')
print('True Sensor Location: {}'.format(s1))
print('SDP optimal sensor 1 location : {}'.format(x_1_star))
print('SDP optimal sensor 2 location : {}'.format(x_2_star))

Results for Sensor Inside of Convex Hull of Anchors
---------------------------------------------------
True Sensor Location: [[ 0.    0.1 ]
 [-0.15  1.8 ]]
SDP optimal sensor 1 location : [0.0, -0.09177517898870537]
SDP optimal sensor 2 location : [0.18698586653590854, 1.5592958096172085]


###Notes
min Trace did not help
mother fucking max trace coming through FTW
